<a href="https://colab.research.google.com/github/manojmrityunjay/AmritaUniversityProjects/blob/main/XIOT_mj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cb-en-p2aie22006-manojvs:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5313186%2F8830490%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240702%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240702T183417Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0e5ac50dfc6e57d723db42e412e3687083341c711ee40a3cb7603c5f5e8df61d9f769b0bc37636342c71bf4cdfbc0afad092d7b7e09ac7675a3ad2ce220e92c99fc3b97fd0e10c2a45e07e4abee41ea12c5e79d04b9f253bfe04b76db180e248a93d170715f93d241ae06752a1c011d36eb1be6de6a3640d493d9221e79a6bfebc8b940976e7dd0b185941498d5134ba93fc686dd444cf4275819e299a4dba80f1928b9f740a87510c82228b0696967b03fe2abdfed0653d64d057ae68be6984e19d51afd8ed39ffc85399e754dba465a794336a6c494f5f16388345c236102c8cc1e5f9268966406885512c241ab2534e817bf36bd643c723081ea5f7607e15'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 114840957 bytes downloaded
Downloaded and uncompressed: cb-en-p2aie22006-manojvs
Data source import complete.


**IMPORT LIBRARIES**

In [2]:
import warnings

# Filter DeprecationWarnings to avoid cluttering output
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Filter DtypeWarnings that might arise from mismatched data types
# (e.g., when converting between cuDF and Pandas)
warnings.filterwarnings("ignore", category=FutureWarning)


In [4]:
!pip install pandas numpy matplotlib seaborn tabulate scikit-learn imblearn stable-baselines3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 12.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [6]:
!pip install cupy-cuda11x --pre -U -f https://pip.cupy.dev/pre

Looking in links: https://pip.cupy.dev/pre
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 MB 9.6 MB/s eta 0:00:00


In [7]:
!nvidia-smi

Tue Jul  2 18:42:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
# !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x Miniconda3-latest-Linux-x86_64.sh
# !bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# !conda init


--2024-07-02 18:46:18--  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.176.84, 104.18.177.84, 2606:4700::6812:b054, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.176.84|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh [following]
--2024-07-02 18:46:18--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146836934 (140M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 140.03M  66.3MB/s    in 2.1s    

2024-07-02 18:46:21 (66.3 MB/s) - ‘Minicond

In [15]:
# !conda create -n rapids-env -c rapidsai -c nvidia -c conda-forge rapids=23.02 python=3.8 cudatoolkit=11.2 -y
# !conda activate rapids-env


Streaming output truncated to the last 5000 lines.












libcurand-dev-10.3.0 | 53.7 MB   | :  51% 0.5089464485453401/1 [01:31<00:16, 34.24s/it] 














libcusolver-dev-11.4 | 66.3 MB   | :  39% 0.38991073868355575/1 [01:31<00:26, 43.61s/it]















libcurand-dev-10.3.0 | 53.7 MB   | :  59% 0.5909595333966463/1 [01:31<00:09, 23.03s/it]














libcusolver-dev-11.4 | 66.3 MB   | :  45% 0.4492808330329552/1 [01:31<00:16, 30.13s/it] 















libcurand-dev-10.3.0 | 53.7 MB   | :  69% 0.6942029558158439/1 [01:31<00:04, 14.67s/it]














libcusolver-dev-11.4 | 66.3 MB   | :  50% 0.5046458019698951/1 [01:31<00:10, 21.67s/it]















libcurand-dev-10.3.0 | 53.7 MB   | :  78% 0.7762160406671501/1 [01:31<00:02, 10.63s/it]
















libcurand-10.3.0.86  | 53.2 MB   | :   0% 0.00029378014500947795/1 [01:31<86:39:49, 312080.84s/it]














libcusolver-dev-11.4 | 66.3 MB   | :  56% 0.5628379182568064/1 [01:31<00:06, 15.47s/it]
















In [24]:
!conda init

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
no change     /root/.bashrc
No action taken.


In [26]:
!conda activate rapids-env


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



CondaError: Run 'conda init' before 'conda activate'



In [16]:
# dyn_mjpu1.py (Complete IDS Model with ML, RL, and DL on Kaggle)

import cudf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tabulate import tabulate

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, make_scorer, matthews_corrcoef,
)
from datetime import datetime
from stable_baselines3 import PPO
from IPython.display import display



In [5]:
nvidia-smi


Tue Jul  2 18:40:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


# TensorFlow TPU/GPU Setup
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU:", resolver.cluster_spec().as_dict()['worker'])
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default strategy for CPU and single GPU
    print("Running on CPU/GPU")


Running on CPU/GPU


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# ***MODULE-1***

**LOADING DATASET**

In [ ]:

# Load the dataset from Kaggle input path
file_path = "/kaggle/input/cb-en-p2aie22006-manojvs/X-IIoTID dataset.csv"
# Read the dataset using cuDF
xiom1 = cudf.read_csv(file_path)
print("cuDF successfully loaded the dataset!")

**EXPLORAORY DATA ANALYSIS (EDA)**

In [ ]:
# Drop unnecessary columns
xiom1 = xiom1.drop(columns=['Timestamp', 'Date', 'class2', 'class3'])
print(xiom1.shape)


In [ ]:
# Get string columns
string_columns = xiom1.select_dtypes(include='object').columns

print("\nString Columns:")
print(list(string_columns))  # Convert to list directly for printing


In [ ]:
print("\nAll Columns:")
print(list(xiom1.columns))  # Convert to list directly for printing


In [ ]:
import cudf

# Assuming you have a cuDF DataFrame called `xiom1`
# ... (your existing code)

# Count of string columns
num_string_cols = xiom1.select_dtypes(include='object').shape[1]
print("Count of string columns:", num_string_cols)

# Count of all columns
num_all_cols = len(xiom1.columns)
print("Count of all columns:", num_all_cols)




In [ ]:
# Count of numeric columns (excluding boolean)
numeric_columns = xiom1.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 'float32', 'float64']).columns
print("\nNumeric Columns:")
print(list(numeric_columns))

In [ ]:
# Count of numeric columns (excluding boolean)
num_numeric_cols = len(numeric_columns)
print("Count of numeric columns:", num_numeric_cols)

In [ ]:
# Calculate Value Counts Before Filtering , class_counts ---> cc , class_counts_df -->cc_df
cc = xiom1['class1'].value_counts()
print("\nValue counts for each class label in Class1:")
cc_df = cc.reset_index()
cc_df.columns = ["Class", "Count"]
print(cc_df.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))


In [ ]:
# Apply Threshold for Removing Low Sample Attacks
threshold = 0.005 * cc.max()

# Filter classes that meet the threshold , filtered_counts ---> f_c
f_c = cc[cc >= threshold]
print(f"\nFiltered classes with counts above {threshold}:")
print(f_c.to_pandas().to_markdown(numalign="left", stralign="left"))


In [ ]:
# Filter the original DataFrame based on filtered_counts
xiom1_flt = xiom1[xiom1['class1'].isin(f_c.index)]
print(xiom1_flt.shape)

In [ ]:
# Recalculate Value Counts After Filtering class_counts_filtered ---> ccf ,class_counts_filtered_to_pandas ---> ccf_pd
ccf = xiom1_flt['class1'].value_counts()
print("\nValue counts for each filtered class label in Class1:")
ccf_pd = ccf.reset_index()
ccf_pd.columns = ["Class", "Count"]
print(ccf_pd.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))

**PREPROCESSING**

> ***1. LABEL ENCODING***

In [ ]:
# Convert cuDF StringColumn to a list before encoding , string_labels ---> s_lb
s_lb = xiom1_flt['class1'].to_arrow().to_pylist()
LE = LabelEncoder()
y_flt = LE.fit_transform(s_lb)  # Extract values directly
# print(s_lb)
# print_class_counts(pd.DataFrame(s_lb, columns=['class1']).groupby('class1').size().reset_index(name='Count'), verbose=8)  # Show top 8 labels


In [ ]:
# ... (Your previous imports, data loading, etc.)

def print_class_counts(data, LE=None, verbose=5):
    """Prints a limited number of class labels and their counts in a table.

    Args:
        data: Data to be displayed (Series, cuDF, or Pandas DataFrame).
        LE: LabelEncoder object (optional, used if labels are numeric).
        verbose (int): Number of top labels to display (default: 5).
    """
    print(f"\nCounts of Samples per Class (Top {verbose}):")

    # Convert series to dataframe if needed
    if isinstance(data, cudf.Series):
        data = data.reset_index(name="Count")
    elif isinstance(data, pd.Series):
        data = data.reset_index(name="Count")

    # Convert labels back to original class names if they are encoded
    if LE is not None and data.iloc[:, 0].dtype == 'int64':
        data.iloc[:, 0] = LE.inverse_transform(data.iloc[:, 0].tolist())

    # Convert to Pandas DataFrame if needed
    if isinstance(data, cudf.DataFrame):
        data = data.to_pandas()

    # Renaming the columns ONLY when the data has only one column (from cudf Series conversion)
    if(len(data.columns) == 1):
        data.columns = ["Class", "Count"]

    # Display the DataFrame (Top verbose classes)
    display(data.head(verbose))

# ... (Previous code for LE, s_lb remains the same)

# Display the s_lb label counts
print_class_counts(cudf.Series(s_lb).value_counts(), verbose=8)  # Convert s_lb to Series and display counts

# # Apply Threshold for Removing Low Sample Attacks
# threshold = 0.005 * cc.max()
# f_c = cc[cc >= threshold]

print(f"\nFiltered classes with counts above {threshold}:")

# Directly display filtered class counts
print_class_counts(f_c, verbose=15)  # Use verbose to limit output


# Display ONLY the filtered class counts using ccf
print_class_counts(xiom1_flt['class1'].value_counts(), LE, verbose=15)


> ***2 .One hot encoding***

In [ ]:
# One-Hot Encode Categorical Features (excluding class1) - Using GPU
OHE = OneHotEncoder()
X_flt = OHE.fit_transform(xiom1_flt.drop(columns=['class1']))  # cuDF DataFrame remains in GPU


In [ ]:
# ... (Your code for importing, data loading, filtering, label encoding)
from imblearn.over_sampling import ADASYN
from scipy.sparse import vstack  # For stacking sparse matrices vertically

In [ ]:
# ... (Your code for importing, data loading, filtering, label encoding)

from imblearn.over_sampling import ADASYN
from scipy.sparse import vstack, coo_matrix

# ... (Rest of your code)

# Resampling (ADASYN):
# Resample with ADASYN DIRECTLY on the Sparse matrix in smaller chunks

# Determine chunk size dynamically based on available memory and sample size
chunk_size = xiom1_flt.shape[0] // 10  # Process in 10 chunks

X_res_chunks = []
y_res_chunks = []

for i in range(0, xiom1_flt.shape[0], chunk_size):  # Use the full shape of xiom1_flt

    # Determine the start and end indices of the chunk
    start = i  # Correct starting index
    end = start + chunk_size

    # Use boolean indexing with CuPy arrays
    X_coo = X_flt.tocoo()
    chunk_rows_mask = (X_coo.row >= start) & (X_coo.row < end)

    chunk_data = X_coo.data[chunk_rows_mask]
    chunk_rows = X_coo.row[chunk_rows_mask] - start  # Adjust row indices to local chunk
    chunk_cols = X_coo.col[chunk_rows_mask]


    # Construct X_chunk_coo from the subset data
    X_chunk_coo = coo_matrix((chunk_data.get(), (chunk_rows.get(), chunk_cols.get())))

    # Get the corresponding labels for this chunk
    y_chunk = y_flt[start:end]

    # Apply ADASYN directly on the sparse matrix
    adasyn = ADASYN(random_state=42)
    X_res_chunk_coo, y_res_chunk = adasyn.fit_resample(X_chunk_coo, y_chunk)

    X_res_chunks.append(X_res_chunk_coo)
    y_res_chunks.append(y_res_chunk)

# Combine the resampled chunks sparse matrices
X_res_coo = vstack(X_res_chunks)
y_res = np.concatenate(y_res_chunks)

# Now convert X_res back to cuDF (from sparse COO)
X_res = cudf.DataFrame.sparse.from_scipy(X_res_coo)

# Split Data (Note that we are splitting the oversampled data now)
X_train, X_temp, y_train, y_temp = train_test_split(X_res, y_res, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
print(f"X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}")
print(f"X_val.shape: {X_val.shape}, y_val.shape: {y_val.shape}")
print(f"X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}")

# ... (Rest of your code: scaling, printing, saving data)


In [ ]:



# # Split Data
# X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Feature Scaling (MinMaxScaler)
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train.to_pandas())
# X_val_scaled = scaler.transform(X_val.to_pandas())
# X_test_scaled = scaler.transform(X_test.to_pandas())

# # Convert back to cudf Dataframes
# X_train_scaled = cudf.DataFrame(X_train_scaled)
# X_val_scaled = cudf.DataFrame(X_val_scaled)
# X_test_scaled = cudf.DataFrame(X_test_scaled)

# # Create Directories to Save Datasets
# base_dir = "/kaggle/working/cb-en-p2aie22006"
# os.makedirs(base_dir, exist_ok=True)

# train_dir = os.path.join(base_dir, "train_data")
# val_dir = os.path.join(base_dir, "val_data")
# test_dir = os.path.join(base_dir, "test_data")
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# # Save Train, Validation, and Test Data
# X_train_scaled.to_csv(os.path.join(train_dir, "train_features.csv"), index=False)
# np.savetxt(os.path.join(train_dir, "train_labels.csv"), y_train, delimiter=",")

# X_val_scaled.to_csv(os.path.join(val_dir, "val_features.csv"), index=False)
# np.savetxt(os.path.join(val_dir, "val_labels.csv"), y_val, delimiter=",")

# X_test_scaled.to_csv(os.path.join(test_dir, "test_features.csv"), index=False)
# np.savetxt(os.path.join(test_dir, "test_labels.csv"), y_test, delimiter=",")



In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
# from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoarddef create_cnn_model(input_shape):
#     model = Sequential([
#         Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
#         MaxPooling1D(pool_size=2),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(64, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')  # Binary classification
#     ])
#     return model

# # Create and train the base model
# with strategy.scope():
#     model = create_cnn_model(input_shape=(X_train_union.shape[1], 1))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model_checkpoint = ModelCheckpoint(
#         filepath="/kaggle/working/best_model_dl.h5",
#         save_best_only=True,
#         monitor="val_accuracy",
#         mode="max",
#         verbose=1
#     )

#     model.fit(
#         X_train_union.to_pandas(), y_train, epochs=10, batch_size=32,
#         validation_data=(X_val_union.to_pandas(), y_val), callbacks=[model_checkpoint, tensorboard_callback]
#     )
#     best_model_dl = load_model("/kaggle/working/best_model_dl.h5")

#     # Transfer Learning Model (VGG16) - (Trained on X_train_union)
#     # Reshape for VGG16 (assuming 128x128 input)
#     X_train_union_reshaped = X_train_union.to_pandas().to_numpy()
#     X_train_union_reshaped = tf.image.resize(X_train_union_reshaped, [128, 128])
#     X_train_union_reshaped = tf.repeat(X_train_union_reshaped[..., np.newaxis], 3, -1)
#     X_val_union_reshaped = X_val_union.to_pandas().to_numpy()
#     X_val_union_reshaped = tf.image.resize(X_val_union_reshaped, [128, 128])
#     X_val_union_reshaped = tf.repeat(X_val_union_reshaped[..., np.newaxis], 3, -1)

#     # Load pre-trained model (VGG16) for transfer learning
#     base_model = tf.keras.applications.VGG16(
#         weights='imagenet', include_top=False, input_shape=(128, 128, 3)
#     )
#     base_model.trainable = False

#     # Flatten the output of the base model
#     x = Flatten()(base_model.output)

#     # Add new classifier layers on top of the base model
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.5)(x)
#     predictions = Dense(1, activation='sigmoid')(x)  # Binary classification

#     # Create the transfer learning model
#     model_transfer = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

#     model_transfer.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     model_transfer_history = model_transfer.fit(
#         X_train_union_reshaped, y_train, epochs=10, batch_size=32,
#         validation_data=(X_val_union_reshaped, y_val)
#     )



# # Evaluate All Models
# models = models_union + models_rl + [best_model_dl, model_transfer]
# model_names = model_names_union + model_names_rl + ['Deep Learning (Basic)', 'Deep Learning (Transfer)']

# specific_attacks = ['C&C', 'Modbus/TCP', 'DoS']

# for i, model in enumerate(models):
#     if isinstance(model, Sequential):  # Check if it's a deep learning model
#         X_test_np = X_test_union.to_numpy()
#         evaluate_and_plot(model, X_test_np, y_test, model_names[i], specific_attacks)  # Evaluate on test data
#     else:
#         if "RL" in model_names[i]:
#             X_test_df = X_test_reduced_rl.to_pandas()
#         else:
#             X_test_df = X_test_union.to_pandas()
#         evaluate_and_plot(model, X_test_df, y_test, model_names[i], specific_attacks)  # Evaluate on test data


# # 8. Additional Analysis (Best Model Selection and Final Evaluation)
# # ------------------------------------------------------------------

# # Lists to store all models and their names (union, rl, and DL)
# all_models = models_union + models_rl
# all_model_names = model_names_union + model_names_rl
# all_x_values = [X_val_union.to_pandas()] * 3 + [X_val_reduced_rl.to_pandas()] * 3


# all_model_accuracies = [
#     grid_search_rf.best_score_,
#     grid_search_et.best_score_,
#     voting_clf_union.score(X_val_union.to_pandas(), y_val),
#     max(model_history.history['val_accuracy']),
#     max(model_transfer_history.history['val_accuracy']),
#     grid_search_rf_rl.best_score_,
#     grid_search_et_rl.best_score_,
#     voting_clf_rl.score(X_val_reduced_rl.to_pandas(), y_val) # Convert to pandas
# ]

# best_model_idx = np.argmax(all_model_accuracies)
# best_model = models[best_model_idx]
# best_model_name = model_names[best_model_idx]

# print(f"\nBest Model based on Validation Accuracy: {best_model_name}")
# print(f"\nBest Model's Validation Accuracy: {np.max(all_model_accuracies)}")
# print(f"\nBest Model's Test Accuracy: {all_model_accuracies[best_model_idx]}")


# # dyn_mjpu5.py (Final Evaluation and Model Saving)
# # ... (Previous code for model training and evaluation)

# # Final Evaluation on Test Set
# if best_model_name in ['Deep Learning (Basic)', 'Deep Learning (Transfer)']:
#     y_pred_best = (best_model.predict(X_test_union_np) > 0.5).astype(int)
# else:
#     X_test_best = (
#         X_test_scaled if best_model_idx < 3 else X_test_reduced_rl
#     )  # Choose the right test set based on the best model
#     y_pred_best = best_model.predict(X_test_best.to_pandas())

# print(f"\nFinal Test Set Classification Report ({best_model_name}):\n", classification_report(y_test, y_pred_best, target_names=le.classes_))

# # Print best parameters for all models (union and RL)
# print(f"\nBest Random Forest Parameters: {grid_search_rf.best_params_}")
# print(f"\nBest Extra Trees Parameters: {grid_search_et.best_params_}")
# print(f"\nBest Random Forest (RL) Parameters: {grid_search_rf_rl.best_params_}")
# print(f"\nBest Extra Trees (RL) Parameters: {grid_search_et_rl.best_params_}")


# # Save the best model to Kaggle Output
# if isinstance(best_model, Sequential):
#     best_model.save("best_model.h5")
# else:
#     import joblib
#     joblib.dump(best_model, 'best_model.pkl')



In [ ]:
# # dyn_mjpu5_pt4.py (Deep Learning and Final Evaluation)
# # ... (all imports from dyn_mjpu5_pt1.py and dyn_mjpu5_pt2.py)
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
# from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


# # 7. Deep Learning Model Training and Evaluation on Test Data
# # ------------------------------------------------------------

# # Define deep learning models with transfer learning and without
# def create_cnn_model(input_shape):
#     model = Sequential([
#         Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
#         MaxPooling1D(pool_size=2),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(64, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')  # Binary classification
#     ])
#     return model

# # Create and train the base model
# with strategy.scope():
#     model = create_cnn_model(input_shape=(X_train_scaled.shape[1], 1))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     # ModelCheckpoint callback to save the best model based on validation accuracy
#     model_checkpoint = ModelCheckpoint(
#         filepath="/kaggle/working/best_model_dl.h5",
#         save_best_only=True,
#         monitor="val_accuracy",
#         mode="max",
#         verbose=1
#     )

#     model.fit(
#         X_train_scaled.to_pandas(), y_train, epochs=10, batch_size=32,
#         validation_data=(X_val_scaled.to_pandas(), y_val), callbacks=[model_checkpoint, tensorboard_callback]
#     )
#     best_model_dl = load_model("/kaggle/working/best_model_dl.h5")

#     # Transfer Learning Model (VGG16) - (Trained on X_train_union)
#     # Reshape for VGG16 (assuming 128x128 input)
#     X_train_union_reshaped = X_train_union.to_pandas().to_numpy()
#     X_train_union_reshaped = tf.image.resize(X_train_union_reshaped, [128, 128])
#     X_train_union_reshaped = tf.repeat(X_train_union_reshaped[..., np.newaxis], 3, -1)
#     X_val_union_reshaped = X_val_union.to_pandas().to_numpy()
#     X_val_union_reshaped = tf.image.resize(X_val_union_reshaped, [128, 128])
#     X_val_union_reshaped = tf.repeat(X_val_union_reshaped[..., np.newaxis], 3, -1)

#     # Load pre-trained VGG16 model
#     base_model = tf.keras.applications.VGG16(
#         weights='imagenet', include_top=False, input_shape=(128, 128, 3)
#     )
#     base_model.trainable = False  # Freeze base model layers

#     model_transfer = Sequential([
#         base_model,
#         Flatten(),
#         Dense(256, activation='relu'),
#         Dropout(0.5),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')  # Binary classification
#     ])

#     model_transfer.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     model_transfer_history = model_transfer.fit(
#         X_train_union_reshaped, y_train, epochs=10, batch_size=32,
#         validation_data=(X_val_union_reshaped, y_val)
#     )



# # 7. Evaluate All Models on the Test Set
# models = models_union + models_rl + [best_model_dl, model_transfer]
# model_names = model_names_union + model_names_rl + ['Deep Learning (Basic)', 'Deep Learning (Transfer)']

# specific_attacks = ['C&C', 'Modbus/TCP', 'DoS']

# for i, model in enumerate(models):
#     if isinstance(model, Sequential):  # Check if it's a deep learning model
#         X_test_np = X_test_scaled.to_numpy()
#         evaluate_and_plot(model, X_test_np, y_test, model_names[i], specific_attacks)  # Evaluate on test data
#     else:
#         if "RL" in model_names[i]:
#             X_test_df = X_test_reduced_rl.to_pandas()
#         else:
#             X_test_df = X_test_union.to_pandas()
#         evaluate_and_plot(model, X_test_df, y_test, model_names[i], specific_attacks)  # Evaluate on test data


# # 8. Additional Analysis (Best Model Selection and Final Evaluation)
# # ------------------------------------------------------------------
# all_model_accuracies = [
#     grid_search_rf.best_score_,
#     grid_search_et.best_score_,
#     voting_clf_union.score(X_val_union.to_pandas(), y_val),
#     max(model_history.history['val_accuracy']),
#     max(model_transfer_history.history['val_accuracy']),
#     grid_search_rf_rl.best_score_,
#     grid_search_et_rl.best_score_,
#     voting_clf_rl.score(X_val_reduced_rl.to_pandas(), y_val) # Convert to pandas
# ]

# best_model_idx = np.argmax(all_model_accuracies)
# best_model = models[best_model_idx]
# best_model_name = model_names[best_model_idx]

# print(f"\nBest Model based on Validation Accuracy: {best_model_name}")
# print(f"\nBest Model's Validation Accuracy: {np.max(all_model_accuracies)}")
# print(f"\nBest Model's Test Accuracy: {all_model_accuracies[best_model_idx]}")


# # Final Evaluation on Test Set
# if best_model_name in ['Deep Learning (Basic)', 'Deep Learning (Transfer)']:
#     y_pred_best = (best_model.predict(X_test_scaled.to_numpy()) > 0.5).astype(int)
# else:
#     X_test_best = (
#         X_test_scaled if best_model_idx < 3 else X_test_reduced_rl
#     )  # Choose the right test set based on the best model
#     y_pred_best = best_model.predict(X_test_best.to_pandas())

# print(f"\nFinal Test Set Classification Report ({best_model_name}):\n", classification_report(y_test, y_pred_best, target_names=le.classes_))

# # Print best parameters for Random Forest and Extra Trees models
# print(f"\nBest Random Forest Parameters: {grid_search_rf.best_params_}")
# print(f"\nBest Extra Trees Parameters: {grid_search_et.best_params_}")
# print(f"\nBest Random Forest (RL features) Parameters: {grid_search_rf_rl.best_params_}")
# print(f"\nBest Extra Trees (RL features) Parameters: {grid_search_et_rl.best_params_}")

# # Save the best model to Kaggle Output
# if isinstance(best_model, Sequential):
#     best_model.save("best_model.h5")
# else:
#     joblib.dump(best_model, 'best_model.pkl')


In [ ]:
#     #dynxpu_skl8
#     import cudf
#     import pandas as pd
#     import numpy as np
#     import matplotlib.pyplot as plt
#     import seaborn as sns
#     import os
#     import joblib  # For saving scikit-learn models

#     from sklearn.model_selection import train_test_split, GridSearchCV
#     from sklearn.preprocessing import LabelEncoder, MinMaxScaler
#     from imblearn.over_sampling import ADASYN
#     from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
#     from sklearn.metrics import (
#         classification_report, confusion_matrix, make_scorer, matthews_corrcoef,
#     )
#     from datetime import datetime
#     from stable_baselines3 import PPO
#     import tensorflow as tf
#     from tensorflow.keras.models import Sequential, load_model
#     from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
#     from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
#     from cuml.preprocessing import OneHotEncoder
#     from cuml.metrics import confusion_matrix

#     # 1. Data Loading and Preprocessing
#     # -----------------------------------
#     # ... [Load the dataset using cuDF, drop unnecessary columns, calculate value counts, apply threshold (optional), and filter the dataset.]
#     # ... [Perform Label Encoding and One-Hot Encode Categorical Features (excluding class1).]
#     # ... [Resample the dataset using ADASYN, then Split Data into train, validation, and test sets.]

#     # Feature Scaling (MinMaxScaler)
#     try:
#         # Use cuML's MinMaxScaler if available
#         from cuml.preprocessing import MinMaxScaler as cuMMScaler
#         scaler = cuMMScaler()
#         X_train_scaled = scaler.fit_transform(X_train)
#         X_val_scaled = scaler.transform(X_val)
#         X_test_scaled = scaler.transform(X_test)

#     except ImportError:
#         # Fallback to scikit-learn's MinMaxScaler if cuML is not available
#         from sklearn.preprocessing import MinMaxScaler

#         scaler = MinMaxScaler()
#         X_train_scaled = scaler.fit_transform(X_train.to_pandas())  # Convert to pandas
#         X_val_scaled = scaler.transform(X_val.to_pandas())       # Convert to pandas
#         X_test_scaled = scaler.transform(X_test.to_pandas())      # Convert to pandas
#         X_train_scaled = cudf.DataFrame(X_train_scaled)
#         X_val_scaled = cudf.DataFrame(X_val_scaled)
#         X_test_scaled = cudf.DataFrame(X_test_scaled)


#     # Create Directories to Save Datasets
#     base_dir = "/kaggle/working/cb-en-p2aie22006"
#     os.makedirs(base_dir, exist_ok=True)

#     train_dir = os.path.join(base_dir, "train_data")
#     val_dir = os.path.join(base_dir, "val_data")
#     test_dir = os.path.join(base_dir, "test_data")
#     os.makedirs(train_dir, exist_ok=True)
#     os.makedirs(val_dir, exist_ok=True)
#     os.makedirs(test_dir, exist_ok=True)


#     # Save Train, Validation, and Test Data
#     X_train_scaled.to_csv(os.path.join(train_dir, "train_features.csv"), index=False)
#     np.savetxt(os.path.join(train_dir, "train_labels.csv"), y_train, delimiter=",")

#     X_val_scaled.to_csv(os.path.join(val_dir, "val_features.csv"), index=False)
#     np.savetxt(os.path.join(val_dir, "val_labels.csv"), y_val, delimiter=",")

#     X_test_scaled.to_csv(os.path.join(test_dir, "test_features.csv"), index=False)
#     np.savetxt(os.path.join(test_dir, "test_labels.csv"), y_test, delimiter=",")

#     print("Completed scaling")

#     # 2. Function Definitions
#     # -------------------------
#     # ... (get_attack_labels and evaluate_and_plot functions remain the same as in the previous response)

#     # 3. Machine Learning Model Training and Evaluation on Original Features
#     # ----------------------------------------------------------------------
#     # ... (Train and evaluate ML models on the original scaled features - Random Forest, Extra Trees, and Voting Classifier)
#     # ... (Feature Selection based on Union of Top Features from RF and ET)

#     # 4. Retrain and evaluate ML models on the union of features
#     # ---------------------------------------------------------

#     # ... (Retrain and evaluate ML models on the union of features - Random Forest, Extra Trees, and Voting Classifier)

#     # 5. Reinforcement Learning-Based Feature Selection
#     # ---------------------------------------------------

#     # ... (Define FeatureSelectionEnv class)

#     # ... (Set up RL environment, train RL model, get best features)

#     # ... (Train and evaluate ML models on the RL-selected features)

#     # 6. Deep Learning Model Training and Evaluation
#     # -----------------------------------------------

#     # ... (Define create_cnn_model function)

#     with strategy.scope():
#         # Basic Deep Learning Model (Trained on X_train_union)
#         # ... (same as before)

#         # Transfer Learning Model (VGG16) - (Trained on X_train_union)
#         # ... (same as before)

#     # 7. Evaluate All Models on the Test Set
#     # ----------------------------------------

#     # ... (Evaluate all models - ML and DL - on the test set)
#     models = models_union + models_rl + [best_model_dl, model_transfer]
#     model_names = model_names_union + model_names_rl + ['Deep Learning (Basic)', 'Deep Learning (Transfer)']

#     specific_attacks = ['C&C', 'Modbus/TCP', 'DoS']

#     for i, model in enumerate(models):
#         if isinstance(model, Sequential):  # Check if it's a deep learning model
#             X_test_np = X_test_union.to_numpy()
#             evaluate_and_plot(model, X_test_np, y_test, model_names[i], specific_attacks)  # Evaluate on test data
#         else:
#             # Convert cuDF to pandas for scikit-learn models if needed
#             if "RL" in model_names[i]:
#                 X_test_df = X_test_reduced_rl.to_pandas()
#             else:
#                 X_test_df = X_test_union.to_pandas()
#             evaluate_and_plot(model, X_test_df, y_test, model_names[i], specific_attacks)  # Evaluate on test data


#     # 8. Additional Analysis (Best Model Selection and Final Evaluation)
#     # ------------------------------------------------------------------
#     all_model_accuracies = [
#         grid_search_rf.best_score_,
#         grid_search_et.best_score_,
#         voting_clf_union.score(X_val_union.to_pandas(), y_val),
#         max(model_history.history['val_accuracy']),
#         max(model_transfer_history.history['val_accuracy']),
#         grid_search_rf_rl.best_score_,
#         grid_search_et_rl.best_score_,
#         voting_clf_rl.score(X_val_reduced_rl.to_pandas(), y_val) # Convert to pandas
#     ]

#     best_model_idx = np.argmax(all_model_accuracies)
#     best_model = models[best_model_idx]
#     best_model_name = model_names[best_model_idx]

#     print(f"\nBest Model based on Validation Accuracy: {best_model_name}")
#     print(f"\nBest Model's Validation Accuracy: {np.max(all_model_accuracies)}")
#     print(f"\nBest Model's Test Accuracy: {all_model_accuracies[best_model_idx]}")

#     # Final Evaluation on Test Set
#     if best_model_name in ['Deep Learning (Basic)', 'Deep Learning (Transfer)']:
#         y_pred_best = (best_model.predict(X_test_scaled.to_numpy()) > 0.5).astype(int)
#     else:
#         X_test_best = (
#             X_test_scaled if best_model_idx < 3 else X_test_reduced_rl
#         )  # Choose the right test set based on the best model
#         y_pred_best = best_model.predict(X_test_best.to_pandas())

#     print(f"\nFinal Test Set Classification Report ({best_model_name}):\n", classification_report(y_test, y_pred_best, target_names=le.classes_))

#     # Print best parameters for all models (union and RL)
#     # ... (same as before)

#     # Save the best model to Kaggle Output
#     if isinstance(best_model, Sequential):
#         best_model.save("best_model.h5")  # Save Keras/TF model
#     else:
#         joblib.dump(best_model, 'best_model.pkl')  # Save scikit-learn model

In [ ]:
# dyn_mjpu6_pt1
# import cudf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from imblearn.over_sampling import ADASYN
# from cuml.preprocessing import OneHotEncoder
# import warnings
# # TensorFlow TPU/GPU Setup
# try:
#   resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#   tf.config.experimental_connect_to_cluster(resolver)
#   tf.tpu.experimental.initialize_tpu_system(resolver)
#   strategy = tf.distribute.TPUStrategy(resolver)
#   print("Running on TPU:", resolver.cluster_spec().as_dict()['worker'])
# except ValueError:
#   strategy = tf.distribute.get_strategy() # Default strategy for CPU and single GPU
#   print("Running on CPU/GPU")


# # Filter DeprecationWarnings to avoid cluttering output
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# # Filter DtypeWarnings that might arise from mismatched data types
# # (e.g., when converting between cuDF and Pandas)
# warnings.filterwarnings("ignore", category=FutureWarning)

# # 1. Data Loading and Preprocessing
# # -----------------------------------

# # Load the dataset from Kaggle input path
# file_path = "/kaggle/input/cb-en-p2aie22006-manojvs/X-IIoTID dataset.csv"

# # Read the dataset using cuDF
# xiom1 = cudf.read_csv(file_path)
# print("cuDF successfully loaded the dataset!")

# # Drop unnecessary columns
# xiom1 = xiom1.drop(columns=['Timestamp', 'Date', 'class2', 'class3'])
# print(xiom1.shape)

# # Calculate Value Counts Before Filtering
# class_counts = xiom1['class1'].value_counts()
# print("\nValue counts for each class label in Class1:")
# class_counts_df = class_counts.reset_index()
# class_counts_df.columns = ["Class", "Count"]
# print(class_counts_df.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))

# # Apply Threshold for Removing Low Sample Attacks (optional)
# threshold = 0.05 * class_counts.max()

# # Filter classes that meet the threshold
# filtered_counts = class_counts[class_counts >= threshold]
# print(f"\nFiltered classes with counts above {threshold}:")
# print(filtered_counts.to_pandas().to_markdown(numalign="left", stralign="left"))

# # Filter the original DataFrame based on filtered_counts
# xiom1_filtered = xiom1[xiom1['class1'].isin(filtered_counts.index)]

# # Recalculate Value Counts After Filtering
# class_counts_filtered = xiom1_filtered['class1'].value_counts()
# print("\nValue counts for each filtered class label in Class1:")
# class_counts_filtered_df = class_counts_filtered.reset_index()
# class_counts_filtered_df.columns = ["Class", "Count"]
# print(class_counts_filtered_df.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))


# # Label Encoding (include "Normal")
# le = LabelEncoder()
# y_filtered = le.fit_transform(xiom1_filtered['class1'].to_arrow().to_pylist())

# # One-Hot Encode Categorical Features (excluding class1) - Using GPU
# encoder = OneHotEncoder()
# X_filtered = encoder.fit_transform(xiom1_filtered.drop(columns=['class1']))  # cuDF DataFrame remains in GPU

# # Manually drop the first column to avoid multicollinearity
# X_filtered = X_filtered.drop(X_filtered.columns[0])  # Correctly drop the first column using cuDF syntax

# # Resampling (ADASYN):
# adasyn = ADASYN(random_state=42)
# X_resampled, y_resampled = adasyn.fit_resample(X_filtered.to_pandas(), y_filtered)  # Convert to pandas for compatibility
# X_resampled = cudf.DataFrame(X_resampled)  # Convert back to cuDF

# # Split Data
# X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# print(f"X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}")
# print(f"X_val.shape: {X_val.shape}, y_val.shape: {y_val.shape}")
# print(f"X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}")

# # Feature Scaling (MinMaxScaler)
# try:
#     # Use cuML's MinMaxScaler if available
#     from cuml.preprocessing import MinMaxScaler as cuMMScaler
#     scaler = cuMMScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)
#     X_test_scaled = scaler.transform(X_test)

# except ImportError:
#     # Fallback to scikit-learn's MinMaxScaler if cuML is not available
#     from sklearn.preprocessing import MinMaxScaler

#     scaler = MinMaxScaler()
#     X_train_scaled = scaler.fit_transform(X_train.to_pandas())  # Convert to pandas
#     X_val_scaled = scaler.transform(X_val.to_pandas())       # Convert to pandas
#     X_test_scaled = scaler.transform(X_test.to_pandas())      # Convert to pandas
#     X_train_scaled = cudf.DataFrame(X_train_scaled)
#     X_val_scaled = cudf.DataFrame(X_val_scaled)
#     X_test_scaled = cudf.DataFrame(X_test_scaled)


# # Create Directories to Save Datasets
# base_dir = "/kaggle/working/cb-en-p2aie22006"
# os.makedirs(base_dir, exist_ok=True)

# train_dir = os.path.join(base_dir, "train_data")
# val_dir = os.path.join(base_dir, "val_data")
# test_dir = os.path.join(base_dir, "test_data")
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)


# # Save Train, Validation, and Test Data
# X_train_scaled.to_csv(os.path.join(train_dir, "train_features.csv"), index=False)
# np.savetxt(os.path.join(train_dir, "train_labels.csv"), y_train, delimiter=",")

# X_val_scaled.to_csv(os.path.join(val_dir, "val_features.csv"), index=False)
# np.savetxt(os.path.join(val_dir, "val_labels.csv"), y_val, delimiter=",")

# X_test_scaled.to_csv(os.path.join(test_dir, "test_features.csv"), index=False)
# np.savetxt(os.path.join(test_dir, "test_labels.csv"), y_test, delimiter=",")

# print("Completed scaling")



In [ ]:
# # dyn_mjpu1.py (Complete IDS Model with ML, RL, and DL on Kaggle)

# import cudf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from imblearn.over_sampling import ADASYN
# from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
# from sklearn.metrics import (
#     classification_report, confusion_matrix, make_scorer, matthews_corrcoef,
# )
# from datetime import datetime
# from stable_baselines3 import PPO
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
# from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


# # 1. Data Loading and Preprocessing
# # -----------------------------------

# # Load the dataset from Kaggle input path
# file_path = "/kaggle/input/cb-en-p2aie22006/X-IIoTID dataset.csv"
# xiom1 = cudf.read_csv(file_path)  # Use cuDF directly
# print("cuDF successfully loaded the dataset!")

# # ... (Drop unnecessary columns, filter for relevant attacks, and label encoding)

# # ... (One-Hot Encode Categorical Features with cuML)

# # ... (Resample using ADASYN)

# # Split Data
# X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Feature Scaling (MinMaxScaler)
# # ... (Use cuML's MinMaxScaler if available, otherwise convert to Pandas and use scikit-learn's MinMaxScaler)

# # ... (Save Train, Validation, and Test Data)


# # 2. Function Definitions
# # -------------------------
# # Define get_attack_labels(y_true, y_pred, attack_names, le):

# # Define evaluate_and_plot(model, X_test, y_test, title, specific_attacks=None)

# # 3. Machine Learning Model Training and Evaluation on Original Features
# # ----------------------------------------------------------------------
# with strategy.scope():
#     # Define the parameter grid for both RandomForest and ExtraTrees classifiers
#     param_grid = {
#         'n_estimators': [50, 100, 200],
#         'max_depth': [None, 10, 20, 30],
#         'min_samples_split': [2, 5, 10],
#         'min_samples_leaf': [1, 2, 4],
#         'bootstrap': [True, False]
#     }

#     mcc_scorer = make_scorer(matthews_corrcoef)

#     # Lists to store models and their names
#     models_union = []
#     model_names_union = []

#     # Train and evaluate ML models on the original scaled features (X_train_scaled, X_val_scaled)
#     # ... (Use GridSearchCV for hyperparameter tuning with cuML models)

#     # Voting Classifier using models trained on original features
#     # ... (Train and evaluate Voting Classifier on X_train_scaled and X_val_scaled)

#     # Feature Selection based on Union of Top Features from RF and ET
#     # ... (Select top_k features from both models, combine them, create X_train_union, X_val_union, X_test_union)

#     # Retrain and evaluate ML models on the union of features (X_train_union, X_val_union)
#     # ... (Use GridSearchCV for hyperparameter tuning with cuML models)

# # 4. Reinforcement Learning-Based Feature Selection
# # ---------------------------------------------------

# # Define FeatureSelectionEnv class (using X_train_scaled and y_train for RL agent training)
# # ...

# # Set up RL environment
# # ... (using X_train_union and y_train for RL agent training)
# # Build and train RL model for Feature Selection
# # ...
# # Use the best RL-selected features
# # ... (Create X_train_reduced_rl, X_val_reduced_rl, X_test_reduced_rl)

# # 5. Model Training and Evaluation with RL-selected Features (Modified for TPU)
# # -------------------------------------------------------------
# with strategy.scope():

#     # Train and Evaluate Random Forest Model on RL Features
#     # ... (Train on X_train_reduced_rl, evaluate on X_val_reduced_rl)

#     # Train and Evaluate Extra Trees Model on RL Features
#     # ... (Train on X_train_reduced_rl, evaluate on X_val_reduced_rl)

#     # Voting Classifier using models trained on RL-selected features
#     # ... (Train on X_train_reduced_rl, evaluate on X_val_reduced_rl)

# # 6. Deep Learning Model Training and Evaluation
# # -----------------------------------------------
# with strategy.scope():
#     # Basic Deep Learning Model (Trained on X_train_union)
#     # ... (same as before)

#     # Transfer Learning Model (VGG16) - (Trained on X_train_union)
#     # ... (same as before)


# # 7. Evaluate All Models on the Test Set
# # ----------------------------------------
# models = models_union + models_rl + [best_model, model_transfer]
# model_names = model_names_union + model_names_rl + ['Deep Learning (Basic)', 'Deep Learning (Transfer)']

# specific_attacks = ['C&C', 'Modbus/TCP', 'DoS']

# for i, model in enumerate(models):
#     # ... (Evaluate each model on the appropriate test set (X_test or X_test_reduced_rl), using y_test)

# # 8. Additional Analysis (Best Model Selection and Final Evaluation)
# # ------------------------------------------------------------------
# # Get the best model based on validation accuracy from all models (union, RL, and DL)
# # ...

# # Final Evaluation on Test Set
# # ...
# # Print best parameters for all models
# # ...
# # Save the best model to Kaggle Output
# # ...


In [ ]:
# dyn_mjpu1.py (Complete IDS Model with ML, RL, and DL on Kaggle)

# Part 1: Data Loading and Preprocessing
# -----------------------------------
# ... (Load the dataset using cuDF)

# ... (Drop unnecessary columns)
# df = df.drop(columns=['Timestamp', 'Date', 'class2', 'class3'])

# Calculate Value Counts Before Filtering
# ... (same as before)

# Apply Threshold for Removing Low Sample Attacks (optional)
# ... (Apply a threshold to filter out low sample attacks if desired)

# Label Encoding (include "Normal")
# ... (same as before, but ensure "Normal" is included in label encoding)

# One-Hot Encode Categorical Features (excluding class1) - Using GPU
# ... (same as before)

# Resampling (ADASYN):
# ... (same as before)

# Split Data
# X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# ... (Feature Scaling using MinMaxScaler)

# ... (Save Train, Validation, and Test Data)


# 2. Function Definitions
# -------------------------
# Define get_attack_labels(y_true, y_pred, attack_names, le):

# Define evaluate_and_plot(model, X_test, y_test, title, specific_attacks=None):
#     ... (Add logic to filter out "Normal" instances from y_test and y_pred
#          when calculating validation accuracy)

# Part 3: Machine Learning Model Training and Evaluation on Original Features
# ... (Use X_train_scaled, y_train, and X_val_scaled for model training and evaluation)

# Part 4: Feature Selection (Union of Top Features and RL-based)
# ... (Select top_k features from both models, combine them, create X_train_union, X_val_union, X_test_union)
# ... (Use X_train_union and y_train for RL agent training)

# Part 5:  Model Training and Evaluation with Feature Selection
# ... (Retrain and evaluate ML models on the union of features and RL-selected features)

# Part 6: Deep Learning Model Training and Evaluation
# ... (Train and evaluate DL models on X_train_union, X_val_union, and X_test_union)

# Part 7: Evaluate All Models on the Test Set
# ... (Evaluate all models on X_test_union and X_test_reduced_rl as appropriate)


# Part 8: Additional Analysis (Best Model Selection and Final Evaluation)
# ------------------------------------------------------------------
# ... (Select the best model based on validation accuracy EXCLUDING "Normal")
# ... (Final Evaluation on Test Set)
# ... (Print Best Parameters for All Models)
# ... (Save the Best Model to Kaggle Output)



In [ ]:
# # dyn_mjpu6_pt4.py (Deep Learning and Final Evaluation)
# # ... (all imports from dyn_mjpu6_pt1.py and dyn_mjpu6_pt2.py)
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
# from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard


# # 7. Deep Learning Model Training and Evaluation on Test Data
# # ------------------------------------------------------------

# # Define deep learning models with transfer learning and without
# def create_cnn_model(input_shape):
#     model = Sequential([
#         Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
#         MaxPooling1D(pool_size=2),
#         Flatten(),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(64, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')  # Binary classification
#     ])
#     return model

# # Create and train the base model
# with strategy.scope():
#     model = create_cnn_model(input_shape=(X_train_scaled.shape[1], 1))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     # ModelCheckpoint callback to save the best model based on validation accuracy
#     model_checkpoint = ModelCheckpoint(
#         filepath="/kaggle/working/best_model_dl.h5",
#         save_best_only=True,
#         monitor="val_accuracy",
#         mode="max",
#         verbose=1
#     )

#     model.fit(
#         X_train_scaled.to_pandas(), y_train, epochs=10, batch_size=32,
#         validation_data=(X_val_scaled.to_pandas(), y_val), callbacks=[model_checkpoint, tensorboard_callback]
#     )
#     best_model_dl = load_model("/kaggle/working/best_model_dl.h5")

#     # Transfer Learning Model (VGG16) - (Trained on X_train_union)
#     # Reshape for VGG16 (assuming 128x128 input)
#     X_train_union_reshaped = X_train_union.to_pandas().to_numpy()
#     X_train_union_reshaped = tf.image.resize(X_train_union_reshaped, [128, 128])
#     X_train_union_reshaped = tf.repeat(X_train_union_reshaped[..., np.newaxis], 3, -1)
#     X_val_union_reshaped = X_val_union.to_pandas().to_numpy()
#     X_val_union_reshaped = tf.image.resize(X_val_union_reshaped, [128, 128])
#     X_val_union_reshaped = tf.repeat(X_val_union_reshaped[..., np.newaxis], 3, -1)

#     # Load pre-trained VGG16 model
#     base_model = tf.keras.applications.VGG16(
#         weights='imagenet', include_top=False, input_shape=(128, 128, 3)
#     )
#     base_model.trainable = False  # Freeze base model layers

#     model_transfer = Sequential([
#         base_model,
#         Flatten(),
#         Dense(256, activation='relu'),
#         Dropout(0.5),
#         Dense(128, activation='relu'),
#         Dropout(0.5),
#         Dense(1, activation='sigmoid')  # Binary classification
#     ])

#     model_transfer.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#     model_transfer_history = model_transfer.fit(
#         X_train_union_reshaped, y_train, epochs=10, batch_size=32,
#         validation_data=(X_val_union_reshaped, y_val)
#     )



# # 7. Evaluate All Models on the Test Set
# models = models_union + models_rl + [best_model_dl, model_transfer]
# model_names = model_names_union + model_names_rl + ['Deep Learning (Basic)', 'Deep Learning (Transfer)']

# specific_attacks = ['C&C', 'Modbus/TCP', 'DoS']

# for i, model in enumerate(models):
#     if isinstance(model, Sequential):  # Check if it's a deep learning model
#         X_test_np = X_test_scaled.to_numpy()
#         evaluate_and_plot(model, X_test_np, y_test, model_names[i], specific_attacks)  # Evaluate on test data
#     else:
#         if "RL" in model_names[i]:
#             X_test_df = X_test_reduced_rl.to_pandas()
#         else:
#             X_test_df = X_test_union.to_pandas()
#         evaluate_and_plot(model, X_test_df, y_test, model_names[i], specific_attacks)  # Evaluate on test data


# # 8. Additional Analysis (Best Model Selection and Final Evaluation)
# # ------------------------------------------------------------------

# # Lists to store all models and their names (union, rl, and DL)
# all_models = models_union + models_rl
# all_model_names = model_names_union + model_names_rl
# all_x_values = [X_val_union.to_pandas()] * 3 + [X_val_reduced_rl.to_pandas()] * 3


# all_model_accuracies = [
#     grid_search_rf.best_score_,
#     grid_search_et.best_score_,
#     voting_clf_union.score(X_val_union.to_pandas(), y_val),
#     max(model_history.history['val_accuracy']),
#     max(model_transfer_history.history['val_accuracy']),
#     grid_search_rf_rl.best_score_,
#     grid_search_et_rl.best_score_,
#     voting_clf_rl.score(X_val_reduced_rl.to_pandas(), y_val) # Convert to pandas
# ]

# best_model_idx = np.argmax(all_model_accuracies)
# best_model = models[best_model_idx]
# best_model_name = model_names[best_model_idx]

# print(f"\nBest Model based on Validation Accuracy: {best_model_name}")
# print(f"\nBest Model's Validation Accuracy: {np.max(all_model_accuracies)}")
# print(f"\nBest Model's Test Accuracy: {all_model_accuracies[best_model_idx]}")


# # Final Evaluation on Test Set
# if best_model_name in ['Deep Learning (Basic)', 'Deep Learning (Transfer)']:
#     y_pred_best = (best_model.predict(X_test_scaled.to_numpy()) > 0.5).astype(int)
# else:
#     X_test_best = (
#         X_test_scaled if best_model_idx < 3 else X_test_reduced_rl
#     )  # Choose the right test set based on the best model
#     y_pred_best = best_model.predict(X_test_best.to_pandas())

# print(f"\nFinal Test Set Classification Report ({best_model_name}):\n", classification_report(y_test, y_pred_best, target_names=le.classes_))

# # Print best parameters for Random Forest and Extra Trees models
# print(f"\nBest Random Forest Parameters: {grid_search_rf.best_params_}")
# print(f"\nBest Extra Trees Parameters: {grid_search_et.best_params_}")
# print(f"\nBest Random Forest (RL features) Parameters: {grid_search_rf_rl.best_params_}")
# print(f"\nBest Extra Trees (RL features) Parameters: {grid_search_et_rl.best_params_}")

# # Save the best model to Kaggle Output
# if isinstance(best_model, Sequential):
#     best_model.save("best_model.h5")
# else:
#     import joblib
#     joblib.dump(best_model, 'best_model.pkl')


In [ ]:
#<dynm_xpu_1>
# import warnings

# # Filter DeprecationWarnings to avoid cluttering output
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# # Filter DtypeWarnings that might arise from mismatched data types
# # (e.g., when converting between cuDF and Pandas)
# warnings.filterwarnings("ignore", category=FutureWarning)


# import cudf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from imblearn.over_sampling import ADASYN
# from cuml.preprocessing import OneHotEncoder
# import warnings
# # TensorFlow TPU/GPU Setup
# try:
#   resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#   tf.config.experimental_connect_to_cluster(resolver)
#   tf.tpu.experimental.initialize_tpu_system(resolver)
#   strategy = tf.distribute.TPUStrategy(resolver)
#   print("Running on TPU:", resolver.cluster_spec().as_dict()['worker'])
# except ValueError:
#   strategy = tf.distribute.get_strategy() # Default strategy for CPU and single GPU
#   print("Running on CPU/GPU")


# # Filter DeprecationWarnings to avoid cluttering output
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# # Filter DtypeWarnings that might arise from mismatched data types
# # (e.g., when converting between cuDF and Pandas)
# warnings.filterwarnings("ignore", category=FutureWarning)

# # 1. Data Loading and Preprocessing
# # -----------------------------------

# # Load the dataset from Kaggle input path
# file_path = "/kaggle/input/cb-en-p2aie22006-manojvs/X-IIoTID dataset.csv"

# # Read the dataset using cuDF
# xiom1 = cudf.read_csv(file_path)
# print("cuDF successfully loaded the dataset!")

# # Drop unnecessary columns
# xiom1 = xiom1.drop(columns=['Timestamp', 'Date', 'class2', 'class3'])
# print(xiom1.shape)

# # Calculate Value Counts Before Filtering
# class_counts = xiom1['class1'].value_counts()
# print("\nValue counts for each class label in Class1:")
# class_counts_df = class_counts.reset_index()
# class_counts_df.columns = ["Class", "Count"]
# print(class_counts_df.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))

# # Apply Threshold for Removing Low Sample Attacks (optional)
# threshold = 0.05 * class_counts.max()

# # Filter classes that meet the threshold
# filtered_counts = class_counts[class_counts >= threshold]
# print(f"\nFiltered classes with counts above {threshold}:")
# print(filtered_counts.to_pandas().to_markdown(numalign="left", stralign="left"))

# # Filter the original DataFrame based on filtered_counts
# xiom1_filtered = xiom1[xiom1['class1'].isin(filtered_counts.index)]

# # Recalculate Value Counts After Filtering
# class_counts_filtered = xiom1_filtered['class1'].value_counts()
# print("\nValue counts for each filtered class label in Class1:")
# class_counts_filtered_df = class_counts_filtered.reset_index()
# class_counts_filtered_df.columns = ["Class", "Count"]
# print(class_counts_filtered_df.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))


# # Label Encoding (include "Normal")
# le = LabelEncoder()
# y_filtered = le.fit_transform(xiom1_filtered['class1'].to_arrow().to_pylist())

# # One-Hot Encode Categorical Features (excluding class1) - Using GPU
# encoder = OneHotEncoder()
# X_filtered = encoder.fit_transform(xiom1_filtered.drop(columns=['class1']))  # cuDF DataFrame remains in GPU

# # Manually drop the first column to avoid multicollinearity
# X_filtered = X_filtered.drop(X_filtered.columns[0])  # Correctly drop the first column using cuDF syntax

# # Resampling (ADASYN):
# adasyn = ADASYN(random_state=42)
# X_resampled, y_resampled = adasyn.fit_resample(X_filtered.to_pandas(), y_filtered)  # Convert to pandas for compatibility
# X_resampled = cudf.DataFrame(X_resampled)  # Convert back to cuDF

# # Split Data
# X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
# print(f"X_train.shape: {X_train.shape}, y_train.shape: {y_train.shape}")
# print(f"X_val.shape: {X_val.shape}, y_val.shape: {y_val.shape}")
# print(f"X_test.shape: {X_test.shape}, y_test.shape: {y_test.shape}")

# # Feature Scaling (MinMaxScaler)
# try:
#     # Use cuML's MinMaxScaler if available
#     from cuml.preprocessing import MinMaxScaler as cuMMScaler
#     scaler = cuMMScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_val_scaled = scaler.transform(X_val)
#     X_test_scaled = scaler.transform(X_test)

# except ImportError:
#     # Fallback to scikit-learn's MinMaxScaler if cuML is not available
#     from sklearn.preprocessing import MinMaxScaler

#     scaler = MinMaxScaler()
#     X_train_scaled = scaler.fit_transform(X_train.to_pandas())  # Convert to pandas
#     X_val_scaled = scaler.transform(X_val.to_pandas())       # Convert to pandas
#     X_test_scaled = scaler.transform(X_test.to_pandas())      # Convert to pandas
#     X_train_scaled = cudf.DataFrame(X_train_scaled)
#     X_val_scaled = cudf.DataFrame(X_val_scaled)
#     X_test_scaled = cudf.DataFrame(X_test_scaled)


# # Create Directories to Save Datasets
# base_dir = "/kaggle/working/cb-en-p2aie22006"
# os.makedirs(base_dir, exist_ok=True)

# train_dir = os.path.join(base_dir, "train_data")
# val_dir = os.path.join(base_dir, "val_data")
# test_dir = os.path.join(base_dir, "test_data")
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)


# # Save Train, Validation, and Test Data
# X_train_scaled.to_csv(os.path.join(train_dir, "train_features.csv"), index=False)
# np.savetxt(os.path.join(train_dir, "train_labels.csv"), y_train, delimiter=",")

# X_val_scaled.to_csv(os.path.join(val_dir, "val_features.csv"), index=False)
# np.savetxt(os.path.join(val_dir, "val_labels.csv"), y_val, delimiter=",")

# X_test_scaled.to_csv(os.path.join(test_dir, "test_features.csv"), index=False)
# np.savetxt(os.path.join(test_dir, "test_labels.csv"), y_test, delimiter=",")

# print("Completed scaling")



In [ ]:
# #<dynamic_xiiot_pt1>
# import warnings

# # Filter DeprecationWarnings to avoid cluttering output
# warnings.filterwarnings("ignore", category=DeprecationWarning)

# # Filter DtypeWarnings that might arise from mismatched data types
# # (e.g., when converting between cuDF and Pandas)
# warnings.filterwarnings("ignore", category=FutureWarning)

# import cudf
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# from tabulate import tabulate

# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from imblearn.over_sampling import ADASYN
# from cuml.preprocessing import OneHotEncoder
# from sklearn.ensemble import VotingClassifier, RandomForestClassifier, ExtraTreesClassifier
# from sklearn.metrics import (
#     classification_report, confusion_matrix, make_scorer, matthews_corrcoef,
# )
# from datetime import datetime
# from stable_baselines3 import PPO


# # TensorFlow TPU/GPU Setup
# try:
#     resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
#     tf.config.experimental_connect_to_cluster(resolver)
#     tf.tpu.experimental.initialize_tpu_system(resolver)
#     strategy = tf.distribute.TPUStrategy(resolver)
#     print("Running on TPU:", resolver.cluster_spec().as_dict()['worker'])
# except ValueError:
#     strategy = tf.distribute.get_strategy()  # Default strategy for CPU and single GPU
#     print("Running on CPU/GPU")

#     # Load the dataset from Kaggle input path
# file_path = "/kaggle/input/cb-en-p2aie22006-manojvs/X-IIoTID dataset.csv"
# # Read the dataset using cuDF
# xiom1 = cudf.read_csv(file_path)
# print("cuDF successfully loaded the dataset!")

# # Drop unnecessary columns
# xiom1 = xiom1.drop(columns=['Timestamp', 'Date', 'class2', 'class3'])
# print(xiom1.shape)

# # Get string columns
# string_columns = xiom1.select_dtypes(include='object').columns

# print("\nString Columns:")
# print(list(string_columns))  # Convert to list directly for printing

# print("\nAll Columns:")
# print(list(xiom1.columns))  # Convert to list directly for printing

# import cudf

# # Assuming you have a cuDF DataFrame called `xiom1`
# # ... (your existing code)

# # Count of string columns
# num_string_cols = xiom1.select_dtypes(include='object').shape[1]
# print("Count of string columns:", num_string_cols)

# # Count of all columns
# num_all_cols = len(xiom1.columns)
# print("Count of all columns:", num_all_cols)

# # Count of numeric columns (excluding boolean)
# numeric_columns = xiom1.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 'float32', 'float64']).columns
# print("\nNumeric Columns:")
# print(list(numeric_columns))

# # Count of numeric columns (excluding boolean)
# num_numeric_cols = len(numeric_columns)
# print("Count of numeric columns:", num_numeric_cols)


# # Calculate Value Counts Before Filtering , class_counts ---> cc , class_counts_df -->cc_df
# cc = xiom1['class1'].value_counts()
# print("\nValue counts for each class label in Class1:")
# cc_df = cc.reset_index()
# cc_df.columns = ["Class", "Count"]
# print(cc_df.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))

# # Apply Threshold for Removing Low Sample Attacks
# threshold = 0.005 * cc.max()

# # Filter classes that meet the threshold , filtered_counts ---> f_c
# f_c = cc[cc >= threshold]
# print(f"\nFiltered classes with counts above {threshold}:")
# print(f_c.to_pandas().to_markdown(numalign="left", stralign="left"))

# # Filter the original DataFrame based on filtered_counts
# xiom1_flt = xiom1[xiom1['class1'].isin(f_c.index)]
# print(xiom1_flt.shape)

# # Recalculate Value Counts After Filtering class_counts_filtered ---> ccf ,class_counts_filtered_to_pandas ---> ccf_pd
# ccf = xiom1_flt['class1'].value_counts()
# print("\nValue counts for each filtered class label in Class1:")
# ccf_pd = ccf.reset_index()
# ccf_pd.columns = ["Class", "Count"]
# print(ccf_pd.to_pandas().to_markdown(index=False, numalign="left", stralign="left"))

# # Convert cuDF StringColumn to a list before encoding , string_labels ---> s_lb
# s_lb = xiom1_flt['class1'].to_arrow().to_pylist()
# LE = LabelEncoder()
# y_flt = LE.fit_transform(s_lb)  # Extract values directly

# # Display a limited number of unique labels and their counts using cudf.Series.value_counts()
# def print_class_counts(cc, LE=None, verbose=5):
#     """Prints a limited number of class labels and their counts in a table.

#     Args:
#         cc: cuDF Series of class labels and counts.
#         LE: LabelEncoder object (optional, used if labels are numeric).
#         verbose (int): Number of top labels to display (default: 5).
#     """
#     from tabulate import tabulate

#     print("\nCounts of Samples per Class (Top {}):".format(verbose))

#     # Check if input is a Series or DataFrame
#     if isinstance(cc, cudf.DataFrame):
#         cc_df = cc
#     elif isinstance(cc, cudf.Series):
#         cc_df = cc.reset_index(name='Count')
#         cc_df = cc_df.rename(columns={'index': 'Class'})


#     # Convert labels back to original class names if they are encoded
#     if LE is not None and cc_df['Class'].dtype=='int64':
#         cc_df['Class'] = cc_df['Class'].to_pandas().apply(lambda x: LE.inverse_transform([x])[0])

#     # Convert to Pandas DataFrames for using tabulate
#     cc_df = cc_df.to_pandas()
#     cc_df = cc_df[['Class', 'Count']]  # Reorder columns

#     # Display using tabulate
#     print(tabulate(cc_df.head(verbose), headers='keys', tablefmt='pipe', showindex=False))

# #Example Usage:
# # Print Class Counts (Top 5 by default)
# print_class_counts(xiom1_flt["class1"].value_counts(), LE)

# # Print Labels (Top 8 by default)
# label_counts = pd.DataFrame(s_lb, columns=['class1']).groupby('class1').size().reset_index(name='Count')
# print_class_counts(label_counts, verbose=8)  # Show top 8 labels

# # Resampling (ADASYN):
# adasyn = ADASYN(random_state=42)
# X_resampled, y_resampled = adasyn.fit_resample(X_filtered.to_pandas(), y_filtered)  # Convert to pandas for compatibility
# X_resampled = cudf.DataFrame(X_resampled)  # Convert back to cuDF

# # Split Data
# X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Feature Scaling (MinMaxScaler)
# scaler = MinMaxScaler()
# X_train_scaled = scaler.fit_transform(X_train.to_pandas())
# X_val_scaled = scaler.transform(X_val.to_pandas())
# X_test_scaled = scaler.transform(X_test.to_pandas())

# # Convert back to cudf Dataframes
# X_train_scaled = cudf.DataFrame(X_train_scaled)
# X_val_scaled = cudf.DataFrame(X_val_scaled)
# X_test_scaled = cudf.DataFrame(X_test_scaled)

# # Create Directories to Save Datasets
# base_dir = "/kaggle/working/cb-en-p2aie22006"
# os.makedirs(base_dir, exist_ok=True)

# train_dir = os.path.join(base_dir, "train_data")
# val_dir = os.path.join(base_dir, "val_data")
# test_dir = os.path.join(base_dir, "test_data")
# os.makedirs(train_dir, exist_ok=True)
# os.makedirs(val_dir, exist_ok=True)
# os.makedirs(test_dir, exist_ok=True)

# # Save Train, Validation, and Test Data
# X_train_scaled.to_csv(os.path.join(train_dir, "train_features.csv"), index=False)
# np.savetxt(os.path.join(train_dir, "train_labels.csv"), y_train, delimiter=",")

# X_val_scaled.to_csv(os.path.join(val_dir, "val_features.csv"), index=False)
# np.savetxt(os.path.join(val_dir, "val_labels.csv"), y_val, delimiter=",")

# X_test_scaled.to_csv(os.path.join(test_dir, "test_features.csv"), index=False)
# np.savetxt(os.path.join(test_dir, "test_labels.csv"), y_test, delimiter=",")
